In [9]:
import numpy as np
import pandas as pd
import random

from pathlib import Path
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GRU, Input, BatchNormalization, Dropout
from ncps.wirings import AutoNCP
from ncps.keras import LTC

# Configuration

In [10]:
NUM_EPOCHS = 200
NUM_EXPERIMENTS = 5

def create_model(train):
    model = Sequential()
    model.add(Input(shape=(train.shape[1], train.shape[2])))

    model.add(LTC(40, return_sequences=True))
    model.add(LTC(30, return_sequences=True))
    model.add(LTC(20, return_sequences=False))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.002, weight_decay=1e-7, use_ema=True), loss='binary_crossentropy', metrics=["accuracy", AUC(name="auc")])
    return model

# Experiment

In [11]:
ID = ["ID"]
USER = ["SubjectID"]
IDS = ["SubjectID", "VideoID"]
TARGET = ["predefinedlabel"]
FEATURES = ["Delta", "Theta", "Alpha1", "Alpha2", "Beta1", "Beta2", "Gamma1", "Gamma2"]
LAGS = []
USE_DIFF = True
INIT_SEED = 5412

In [12]:
data_dir = Path("/home/aseliverstov/projects/brain_signals/data")
data = pd.read_csv(data_dir / "EEG_data.csv")

new_features = []
for lag in LAGS:
    for feature_name in FEATURES:
        new_feature_name = f"{feature_name}_{lag}"
        new_features.append(new_feature_name)
        data[new_feature_name] = data.groupby(IDS)[feature_name].shift(lag).fillna(0)
FEATURES.extend(new_features)

if USE_DIFF:
    for feature_name in FEATURES:
        data[feature_name] = data[feature_name] - data.groupby(IDS)[feature_name].shift(1).fillna(0)

data["ID"] = (len(np.unique(data["VideoID"])) * data["SubjectID"] + data["VideoID"]).astype("int")
data = data[ID + USER + FEATURES + TARGET]

data.head(3)

,ID,SubjectID,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel
0,0,0.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0
1,0,0.0,-228176.0,-62529.0,-32296.0,-21751.0,-25200.0,-41410.0,-27935.0,-5553.0,0.0
2,0,0.0,684566.0,355662.0,200560.0,59867.0,33547.0,126849.0,51950.0,22614.0,0.0


In [13]:
def reshape_dataset(data):
    features = []
    target = []
    for cur_id in np.unique(data[ID].to_numpy()):
        cur_id_data = data[data[ID].to_numpy() == cur_id]
        target.append(np.mean(cur_id_data[TARGET].to_numpy()).astype("int"))
        features.append(cur_id_data[FEATURES].to_numpy())

    features = pad_sequences(features)
    return np.array(features), np.array(target)

def pad_sequences(arrays, pad_value=0):
    max_length = max(arr.shape[0] for arr in arrays)
    padded_arrays = [
        np.pad(
            arr,
            ((0, max_length - arr.shape[0]), (0, 0)),
            mode='constant',
            constant_values=pad_value)
            for arr in arrays
        ]
    return np.stack(padded_arrays)

In [14]:
X, _ = reshape_dataset(data)
model = create_model(X)
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ ltc_24 (LTC)                    │ (None, 144, 40)        │         7,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc_25 (LTC)                    │ (None, 144, 30)        │         8,630 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc_26 (LTC)                    │ (None, 20)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,707 (80.89 KB)

 Trainable params: 20,707 (80.89 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
%%time

all_acc = []
all_loss = []
all_auc = []

all_val_acc = []
all_val_loss = []
all_val_auc = []

for j, seed in tqdm(enumerate(np.arange(NUM_EXPERIMENTS) + INIT_SEED)):
    np.random.seed(int(seed))
    random.seed(int(seed))
    tf.random.set_seed(int(seed))

    train_id = np.random.choice(np.unique(np.ravel(data[USER])), 7, replace=False)
    train_index = np.isin(data[USER], train_id)

    train = data.iloc[train_index]
    test = data.iloc[~train_index]

    X_train, y_train = reshape_dataset(train)
    X_test, y_test = reshape_dataset(test)

    y_train = y_train.reshape(-1, 1)
    y_test = y_test.reshape(-1, 1)

    model = create_model(X_train)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=NUM_EPOCHS,
        batch_size=10,
        verbose=1,
    )

    acc = history.history['accuracy']
    loss = history.history['loss']
    auc = history.history['auc']

    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    val_auc = history.history['val_auc']

    all_acc.append(acc)
    all_loss.append(loss)
    all_auc.append(auc)

    all_val_acc.append(val_acc)
    all_val_loss.append(val_loss)
    all_val_auc.append(val_auc)

epoch_acc = np.mean(all_acc, axis=0)
epoch_loss = np.mean(all_loss, axis=0)
epoch_auc = np.mean(all_auc, axis=0)

epoch_val_acc = np.mean(all_val_acc, axis=0)
epoch_val_loss = np.mean(all_val_loss, axis=0)
epoch_val_auc = np.mean(all_val_auc, axis=0)

0it [00:00, ?it/s]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.4665 - auc: 0.4582 - loss: 0.7548 - val_accuracy: 0.5000 - val_auc: 0.4000 - val_loss: 0.6941
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.4468 - auc: 0.4848 - loss: 0.6960 - val_accuracy: 0.5000 - val_auc: 0.4667 - val_loss: 0.6963
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.5335 - auc: 0.5370 - loss: 0.6910 - val_accuracy: 0.5000 - val_auc: 0.5667 - val_loss: 0.6951
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.5335 - auc: 0.5492 - loss: 0.6901 - val_accuracy: 0.5000 - val_auc: 0.5667 - val_loss: 0.6927
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.5518 - auc: 0.5432 - loss: 0.6916 - val_accuracy: 0.5000 - val_auc: 0.5667 - val_loss: 0.6922
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.4665 - auc: 0.5384 - loss: 0.6936 - val_accuracy: 0.5000 - val_auc: 0.5667 - val_loss: 0.6918
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.4

In [ ]:
for i in range(NUM_EPOCHS):
    print(f"Epoch {(i + 1)}: TRAIN Accuracy = {np.round(epoch_acc[i], 3)} Loss = {np.round(epoch_loss[i], 3)} AUC = {np.round(epoch_auc[i], 3)}")
    print(f"Epoch {(i + 1)}: VAL Accuracy = {np.round(epoch_val_acc[i], 3)} Loss = {np.round(epoch_val_loss[i], 3)} AUC = {np.round(epoch_val_auc[i], 3)}")

with open("./logs/Liquidv3_emb4.txt", "w") as f:
    for i in range(NUM_EPOCHS):
        f.write(f"Epoch {(i + 1)}: TRAIN Accuracy = {np.round(epoch_acc[i], 3)} Loss = {np.round(epoch_loss[i], 3)} AUC = {np.round(epoch_auc[i], 3)}\n")
        f.write(f"Epoch {(i + 1)}: VAL Accuracy = {np.round(epoch_val_acc[i], 3)} Loss = {np.round(epoch_val_loss[i], 3)} AUC = {np.round(epoch_val_auc[i], 3)}\n")

Epoch 1: TRAIN Accuracy = 0.474 Loss = 0.712 AUC = 0.418
Epoch 1: VAL Accuracy = 0.487 Loss = 0.695 AUC = 0.507
Epoch 2: TRAIN Accuracy = 0.517 Loss = 0.695 AUC = 0.503
Epoch 2: VAL Accuracy = 0.5 Loss = 0.694 AUC = 0.513
Epoch 3: TRAIN Accuracy = 0.503 Loss = 0.694 AUC = 0.522
Epoch 3: VAL Accuracy = 0.5 Loss = 0.694 AUC = 0.56
Epoch 4: TRAIN Accuracy = 0.477 Loss = 0.698 AUC = 0.455
Epoch 4: VAL Accuracy = 0.513 Loss = 0.693 AUC = 0.54
Epoch 5: TRAIN Accuracy = 0.449 Loss = 0.698 AUC = 0.446
Epoch 5: VAL Accuracy = 0.52 Loss = 0.693 AUC = 0.593
Epoch 6: TRAIN Accuracy = 0.509 Loss = 0.693 AUC = 0.515
Epoch 6: VAL Accuracy = 0.5 Loss = 0.692 AUC = 0.587
Epoch 7: TRAIN Accuracy = 0.506 Loss = 0.695 AUC = 0.495
Epoch 7: VAL Accuracy = 0.527 Loss = 0.692 AUC = 0.567
Epoch 8: TRAIN Accuracy = 0.497 Loss = 0.694 AUC = 0.514
Epoch 8: VAL Accuracy = 0.52 Loss = 0.692 AUC = 0.607
Epoch 9: TRAIN Accuracy = 0.509 Loss = 0.691 AUC = 0.54
Epoch 9: VAL Accuracy = 0.513 Loss = 0.692 AUC = 0.6
Epoch